In [ ]:
import wandb
import pandas as pd

api = wandb.Api()

DUMMY_RUN = "rap1ide/slice_inflate/xx190fww"

pd.set_option('display.max_rows', 20)
pd.set_option('display.max_columns', 20)
pd.set_option('display.width', 1000)

def get_agg_dict(filtered_frame):
    agg_dict = {
        n: 'first' if t != float else 'mean' for n,t in zip(filtered_frame.columns,filtered_frame.dtypes)
    }
    return agg_dict

# get n chunks of a list
def chunks(lst, n):
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

def join_cols(frame, first_col_key, second_col_key, join_str="+", drop_second=True):
    first_col = frame[first_col_key].astype(str)
    second_col = frame[second_col_key].astype(str)
    frame[first_col_key] = first_col + join_str + second_col
    if drop_second:
        frame.drop(second_col_key, axis=1, inplace=True)
    else:
        frame.drop(first_col_key, axis=1, inplace=True)
    return frame

In [ ]:
settings = {
   "0:1": dict(description="Clinical standard",
      first_view="p2CH",
      second_view="p4CH",
      prescan_res ="$(1.5mm)^3$",
      prescan_type ="GT",
      slice_res ="$(1.5mm)^2$",
      slice_type="GT",
   ),
   "1:2": dict(description="Clinical standard",
      first_view="2CH",
      second_view="4CH",
      prescan_res ="$(1.5mm)^3$",
      prescan_type ="GT",
      slice_res ="$(1.5mm)^2$",
      slice_type="GT",
   ),
   "2:3": dict(description="Clinical standard",
      first_view="2CH",
      second_view="SA",
      prescan_res ="$(1.5mm)^3$",
      prescan_type ="GT",
      slice_res ="$(1.5mm)^2$",
      slice_type="GT",
   ),
   "3:9": dict(description="Mean out of 6 Random",
      first_view="RND",
      second_view="RND",
      prescan_res ="$(1.5mm)^3$",
      prescan_type ="GT",
      slice_res ="$(1.5mm)^2$",
      slice_type="GT",
   ),
   "9:10": dict(description="Optimized",
      first_view="OPT",
      second_view="OPT",
      prescan_res ="$(1.5mm)^3$",
      prescan_type ="GT",
      slice_res ="$(1.5mm)^2$",
      slice_type="GT",
   ),

   "10:11": dict(description="Clinical standard",
      first_view="2CH",
      second_view="4CH",
      prescan_res ="$(6mm)^3$",
      prescan_type ="GT",
      slice_res ="$(1.5mm)^2$",
      slice_type="GT",
   ),
   "11:12": dict(description="Optimized",
      first_view="OPT",
      second_view="OPT",
      prescan_res ="$(6mm)^3$",
      prescan_type ="GT",
      slice_res ="$(1.5mm)^2$",
      slice_type="GT",
   ),

   "12:13": dict(description="Clinical standard",
      first_view="2CH",
      second_view="4CH",
      prescan_res ="$(6mm)^3$",
      prescan_type ="SEG",
      slice_res ="$(1.5mm)^2$",
      slice_type="SEG",
   ),
   "13:14": dict(description="Optimized",
      first_view="OPT",
      second_view="OPT",
      prescan_res ="$(6mm)^3$",
      prescan_type ="SEG",
      slice_res ="$(1.5mm)^2$",
      slice_type="SEG",
   ),

   "14:15": dict(description="Clinical standard",
      first_view="2CH",
      second_view="4CH",
      prescan_res ="$(6mm)^3$",
      prescan_type ="SEG",
      slice_res ="$(6mm)^3$",
      slice_type="SEG",
   ),
   "15:16": dict(description="Optimized",
      first_view="OPT",
      second_view="OPT",
      prescan_res ="$(6mm)^3$",
      prescan_type ="SEG",
      slice_res ="$(6mm)^3$",
      slice_type="SEG",
   ),
}

# Define run paths of wandb MMWHS

In [ ]:
mmwhs_run_path_list = list(reversed([
'rap1ide/slice_inflate/runs/zk9i0rrx',
'rap1ide/slice_inflate/runs/ge0mauw4',
'rap1ide/slice_inflate/runs/t1dl1x69',
'rap1ide/slice_inflate/runs/fjmhxre7',
'rap1ide/slice_inflate/runs/vm0sjun1',
'rap1ide/slice_inflate/runs/p2eoy8z0',
'rap1ide/slice_inflate/runs/9gtmnnfv',
'rap1ide/slice_inflate/runs/hvvddqer',
'rap1ide/slice_inflate/runs/kfzdcepx',
'rap1ide/slice_inflate/runs/5m1et0aq',
'rap1ide/slice_inflate/runs/c4gqhk82',
'rap1ide/slice_inflate/runs/okxvp25d',
'rap1ide/slice_inflate/runs/5jj2gdhk',
'rap1ide/slice_inflate/runs/ez1e3vr3',
'rap1ide/slice_inflate/runs/upt7xjkn',
'rap1ide/slice_inflate/runs/vpce07jm',
'rap1ide/slice_inflate/runs/qmvin2ux',
'rap1ide/slice_inflate/runs/wl068f1l',
'rap1ide/slice_inflate/runs/dkgq89df',
'rap1ide/slice_inflate/runs/4vrszjn4',
'rap1ide/slice_inflate/runs/jiolfg7j',
'rap1ide/slice_inflate/runs/5estavq0',
'rap1ide/slice_inflate/runs/eoc7me0e',
'rap1ide/slice_inflate/runs/8ac6id01',
'rap1ide/slice_inflate/runs/wao2b16c',
'rap1ide/slice_inflate/runs/sih8wc65',
'rap1ide/slice_inflate/runs/pmyls33r',
'rap1ide/slice_inflate/runs/u8bdwaji',
'rap1ide/slice_inflate/runs/lad69u2m',
'rap1ide/slice_inflate/runs/jsw6an38',
'rap1ide/slice_inflate/runs/x5w8hml2',
'rap1ide/slice_inflate/runs/gobs9276',
    'rap1ide/slice_inflate/runs/w0bgwxn3',
    'rap1ide/slice_inflate/runs/6plcdxdr',
    'rap1ide/slice_inflate/runs/e975wi0p',
    'rap1ide/slice_inflate/runs/ukoauy9x',
    'rap1ide/slice_inflate/runs/gd12cq5u',
    'rap1ide/slice_inflate/runs/cjbdy78r',
    'rap1ide/slice_inflate/runs/b8rh2d09',
    'rap1ide/slice_inflate/runs/9xe5ql4h',
    'rap1ide/slice_inflate/runs/cte64bvv',
    'rap1ide/slice_inflate/runs/8vbmnmre',
    'rap1ide/slice_inflate/runs/4te8z4rk',
    'rap1ide/slice_inflate/runs/ay7bgyd5',
    'rap1ide/slice_inflate/runs/sfh7xw2p',
    'rap1ide/slice_inflate/runs/zdpeonni',
    'rap1ide/slice_inflate/runs/j16qb5rm',
    'rap1ide/slice_inflate/runs/4kdmc0ch',
    'rap1ide/slice_inflate/runs/mag0njn8',
    'rap1ide/slice_inflate/runs/6d19um8q',
    'rap1ide/slice_inflate/runs/g5hai89a',
    'rap1ide/slice_inflate/runs/31zvi2a0',
    'rap1ide/slice_inflate/runs/f0mu5sbk',
    'rap1ide/slice_inflate/runs/krljgne0',
    'rap1ide/slice_inflate/runs/n41shvul',
    'rap1ide/slice_inflate/runs/wzmr06qe',
    'rap1ide/slice_inflate/runs/0spvd8jj',
    'rap1ide/slice_inflate/runs/jl7prqo8',
    'rap1ide/slice_inflate/runs/n2rf1ae4',
    'rap1ide/slice_inflate/runs/5im33z1m',
    'rap1ide/slice_inflate/runs/j7h289df',
    'rap1ide/slice_inflate/runs/g85qf8ss',
    'rap1ide/slice_inflate/runs/q3963sr5',
    'rap1ide/slice_inflate/runs/tww9c2fm',
    'rap1ide/slice_inflate/runs/y9m2v1b3',
    'rap1ide/slice_inflate/runs/0snhugvl',
    'rap1ide/slice_inflate/runs/a2go54je',
    'rap1ide/slice_inflate/runs/81zajv9e',
]))

In [ ]:
mmwhs_run_name_list = list(reversed([
'20240126__03_45_52_progressive-granite_stage-2_fold-1',
'20240126__06_34_54_mad-prefix_stage-2_fold-0',
'20240126__03_45_52_progressive-granite_stage-1_fold-1',
'20240126__06_34_54_mad-prefix_stage-1_fold-0',
'20240126__03_45_52_crazy-core_stage-4_fold-0',
'20240125__21_15_50_leather-plot_stage-4_fold-2',
'20240126__03_45_52_crazy-core_stage-3_fold-0',
'20240125__21_15_50_leather-plot_stage-3_fold-2',
'20240126__03_45_52_crazy-core_stage-2_fold-0',
'20240125__21_15_50_leather-plot_stage-2_fold-2',
'20240126__03_45_52_crazy-core_stage-1_fold-0',
'20240125__21_15_50_leather-plot_stage-1_fold-2',
'20240125__21_11_31_agile-directive_stage-4_fold-2',
'20240125__21_11_31_agile-directive_stage-3_fold-2',
'20240125__21_15_50_magenta-glass_stage-4_fold-1',
'20240125__21_11_31_agile-directive_stage-2_fold-2',
'20240125__21_15_50_magenta-glass_stage-3_fold-1',
'20240125__21_11_31_agile-directive_stage-1_fold-2',
'20240125__21_15_50_magenta-glass_stage-2_fold-1',
'20240125__21_11_31_coal-adjugate_stage-4_fold-1',
'20240125__21_15_50_magenta-glass_stage-1_fold-1',
'20240125__21_11_31_coal-adjugate_stage-3_fold-1',
'20240125__21_15_50_plain-mask_stage-4_fold-0',
'20240125__21_11_31_coal-adjugate_stage-2_fold-1',
'20240125__21_11_31_coal-adjugate_stage-1_fold-1',
'20240125__21_15_50_plain-mask_stage-3_fold-0',
'20240125__21_11_31_sticky-league_stage-4_fold-0',
'20240125__21_11_31_sticky-league_stage-3_fold-0',
'20240125__21_15_50_plain-mask_stage-2_fold-0',
'20240125__21_11_31_sticky-league_stage-2_fold-0',
'20240125__21_15_50_plain-mask_stage-1_fold-0',
'20240125__21_11_31_sticky-league_stage-1_fold-0',
    '20240125__17_37_07_active-caramel_stage-1_fold-2',
    '20240125__17_37_07_wide-cube_stage-1_fold-1',
    '20240125__17_37_07_lazy-object_stage-1_fold-0',
    '20240125__15_12_13_plain-pizza_stage-1_fold-2',
    '20240125__15_12_13_chartreuse-limestone_stage-1_fold-1',
    '20240125__15_12_13_human-bracket_stage-1_fold-0',
    '20240125__14_28_50_raw-gravity_stage-1_fold-2',
    '20240125__14_28_50_clever-lion_stage-1_fold-1',
    '20240125__14_28_50_cyan-negation_stage-1_fold-0',
    '20240125__11_31_39_chill-keyframe_stage-1_fold-2',
    '20240125__11_31_39_vibrant-frequency_stage-1_fold-1',
    '20240125__11_31_39_warm-lepton_stage-1_fold-0',
    '20240125__11_03_37_savory-hangar_stage-1_fold-2',
    '20240125__11_03_37_ascent-incubator_stage-1_fold-1',
    '20240125__11_03_37_crunchy-pouf_stage-1_fold-0',
    '20240125__10_35_37_uniform-cycle_stage-1_fold-2',
    '20240125__10_35_37_mean-goal_stage-1_fold-1',
    '20240125__10_35_37_modern-riser_stage-1_fold-0',
    '20240125__10_07_01_nullary-supervisor_stage-1_fold-2',
    '20240125__10_07_01_tangy-bow_stage-1_fold-1',
    '20240125__10_07_01_cold-yeast_stage-1_fold-0',
    '20240125__09_38_16_clever-relish_stage-1_fold-2',
    '20240125__09_38_16_brute-ion_stage-1_fold-1',
    '20240125__09_38_16_rectilinear-level_stage-1_fold-0',
    '20240125__09_10_32_free-inflection_stage-1_fold-2',
    '20240125__09_10_32_wide-crocodile_stage-1_fold-1',
    '20240125__09_10_32_creamy-web_stage-1_fold-0',
    '20240125__08_41_32_celeste-dataframe_stage-1_fold-2',
    '20240125__08_41_32_deafening-filler_stage-1_fold-1',
    '20240125__08_41_32_brownian-chronology_stage-1_fold-0',
    '20240125__08_14_19_furious-asset_stage-1_fold-2',
    '20240125__08_14_19_deterministic-convergence_stage-1_fold-1',
    '20240125__08_14_19_free-editor_stage-1_fold-0',
    '20240125__07_46_26_violent-relativity_stage-1_fold-2',
    '20240125__07_46_26_free-graph_stage-1_fold-1',
    '20240125__07_46_26_trusting-rosin_stage-1_fold-0',
]))

import numpy as np
new_mmwhs_sorted_args = np.argsort(mmwhs_run_name_list)#[36:64]

mmwhs_run_name_list = list(np.array(mmwhs_run_name_list)[new_mmwhs_sorted_args])
mmwhs_run_path_list = list(np.array(mmwhs_run_path_list)[new_mmwhs_sorted_args])

In [ ]:
mmwhs_run_name_list

### Print run lists for convenience MMWHS

In [ ]:
for c_id, (nc, pc) in enumerate(zip(chunks(mmwhs_run_name_list, 4), chunks(mmwhs_run_path_list, 4))):
    print(f"## {c_id}")
    for n, p in zip(nc, pc):
        print(n, "\t", p)
print()


In [ ]:
mmwhs_run_dict = {
   ######## 00 ########
   "20240125__07_46_26_trusting-rosin_stage-1_fold-0": dict(
      access_key="rap1ide/slice_inflate/81zajv9e",
      id=0,
      fold=0,
   ),
   "20240125__07_46_26_free-graph_stage-1_fold-1": dict(
      access_key="rap1ide/slice_inflate/a2go54je",
      id=0,
      fold=1,
   ),
   "20240125__07_46_26_violent-relativity_stage-1_fold-2": dict(
      access_key="rap1ide/slice_inflate/0snhugvl",
      id=0,
      fold=2,
   ),

   ######## 01 ########
   "20240125__08_14_19_free-editor_stage-1_fold-0": dict(
      access_key="rap1ide/slice_inflate/y9m2v1b3",
      id=1,
      fold=0,
   ),
   "20240125__08_14_19_deterministic-convergence_stage-1": dict(
      access_key="rap1ide/slice_inflate/tww9c2fm",
      id=1,
      fold=1,
   ),
   "20240125__08_14_19_furious-asset_stage-1_fold-2": dict(
      access_key="rap1ide/slice_inflate/q3963sr5",
      id=1,
      fold=2,
   ),

   ######## 02 ########
   "20240125__08_41_32_brownian-chronology_stage-1_fold-0": dict(
      access_key="rap1ide/slice_inflate/g85qf8ss",
      id=2,
      fold=0,
   ),
   "20240125__08_41_32_deafening-filler_stage-1_fold-1": dict(
      access_key="rap1ide/slice_inflate/j7h289df",
      id=2,
      fold=1,
   ),
   "20240125__08_41_32_celeste-dataframe_stage-1_fold-2": dict(
      access_key="rap1ide/slice_inflate/5im33z1m",
      id=2,
      fold=2,
   ),

   ######## 03 ########
   "20240125__09_10_32_creamy-web_stage-1_fold-0 ": dict(
      access_key="rap1ide/slice_inflate/n2rf1ae4",
      id=3,
      fold=0,
   ),
   "20240125__09_10_32_wide-crocodile_stage-1_fold-1": dict(
      access_key="rap1ide/slice_inflate/jl7prqo8",
      id=3,
      fold=1,
   ),
   "20240125__09_10_32_free-inflection_stage-1_fold-2": dict(
      access_key="rap1ide/slice_inflate/0spvd8jj",
      id=3,
      fold=2,
   ),

   ######## 04 ########
   "20240125__09_38_16_rectilinear-level_stage-1_fold-0": dict(
      access_key="rap1ide/slice_inflate/wzmr06qe",
      id=4,
      fold=0,
   ),
   "20240125__09_38_16_brute-ion_stage-1_fold-1": dict(
      access_key="rap1ide/slice_inflate/n41shvul",
      id=4,
      fold=1,
   ),
   "20240125__09_38_16_clever-relish_stage-1_fold-2": dict(
      access_key="rap1ide/slice_inflate/krljgne0",
      id=4,
      fold=2,
   ),

   ######## 05 ########
   "20240125__10_07_01_cold-yeast_stage-1_fold-0": dict(
      access_key="rap1ide/slice_inflate/f0mu5sbk",
      id=5,
      fold=0,
   ),
   "20240125__10_07_01_tangy-bow_stage-1_fold-1": dict(
      access_key="rap1ide/slice_inflate/31zvi2a0",
      id=5,
      fold=1,
   ),
   "20240125__10_07_01_nullary-supervisor_stage-1_fold-2": dict(
      access_key="rap1ide/slice_inflate/g5hai89a",
      id=5,
      fold=2,
   ),

   ######## 06 ########
   "20240125__10_35_37_modern-riser_stage-1_fold-0": dict(
      access_key="rap1ide/slice_inflate/6d19um8q",
      id=6,
      fold=0,
   ),
   "20240125__10_35_37_mean-goal_stage-1_fold-1": dict(
      access_key="rap1ide/slice_inflate/mag0njn8",
      id=6,
      fold=1,
   ),
   "20240125__10_35_37_uniform-cycle_stage-1_fold-2": dict(
      access_key="rap1ide/slice_inflate/4kdmc0ch",
      id=6,
      fold=2,
   ),

   ######## 07 ########
   "20240125__11_03_37_crunchy-pouf_stage-1_fold-0": dict(
      access_key="rap1ide/slice_inflate/j16qb5rm",
      id=7,
      fold=0,
   ),
   "20240125__11_03_37_savory-hangar_stage-1_fold-2 ": dict(
      access_key="rap1ide/slice_inflate/zdpeonni",
      id=7,
      fold=1,
   ),
   "20240125__11_03_37_savory-hangar_stage-1_fold-2": dict(
      access_key="rap1ide/slice_inflate/sfh7xw2p",
      id=7,
      fold=2,
   ),
   ######## 08 ########
   "20240125__11_31_39_warm-lepton_stage-1_fold-0": dict(
      access_key="rap1ide/slice_inflate/ay7bgyd5",
      id=8,
      fold=0,
   ),
   "20240125__11_31_39_vibrant-frequency_stage-1_fold-1": dict(
      access_key="rap1ide/slice_inflate/4te8z4rk",
      id=8,
      fold=1,
   ),
   "20240125__11_31_39_chill-keyframe_stage-1_fold-2 ": dict(
      access_key="rap1ide/slice_inflate/8vbmnmre",
      id=8,
      fold=2,
   ),

   ######## 09 ########
   "dummy27": dict(
      access_key=DUMMY_RUN,
      id=9,
      fold=0,
   ),
   "dummy28": dict(
      access_key=DUMMY_RUN,
      id=9,
      fold=1,
   ),
   "dummy29": dict(
      access_key=DUMMY_RUN,
      id=9,
      fold=2,
   ),

   ######## 10 ########
   "20240125__14_28_50_cyan-negation_stage-1_fold-0": dict(
      access_key="rap1ide/slice_inflate/runs/cte64bvv",
      id=10,
      fold=0,
   ),
   "20240125__14_28_50_clever-lion_stage-1_fold-1": dict(
      access_key="rap1ide/slice_inflate/runs/9xe5ql4h",
      id=10,
      fold=1,
   ),
   "20240125__14_28_50_raw-gravity_stage-1_fold-2": dict(
      access_key="rap1ide/slice_inflate/runs/b8rh2d09",
      id=10,
      fold=2,
   ),

   ######## 11 ########
   "20240125__21_11_31_sticky-league_stage-3_fold-0 ": dict(
      access_key="rap1ide/slice_inflate/runs/u8bdwaji",
      id=11,
      fold=0,
   ),
   "20240125__21_11_31_coal-adjugate_stage-3_fold-1": dict(
      access_key="rap1ide/slice_inflate/runs/5estavq0",
      id=11,
      fold=1,
   ),
   "20240125__21_11_31_agile-directive_stage-3_fold-2": dict(
      access_key="rap1ide/slice_inflate/runs/ez1e3vr3",
      id=11,
      fold=2,
   ),
   ######## 12 ########
   "20240125__15_12_13_human-bracket_stage-1_fold-0": dict(
      access_key="rap1ide/slice_inflate/runs/cjbdy78r",
      id=12,
      fold=0,
   ),
   "20240125__15_12_13_chartreuse-limestone_stage-1_fold-1": dict(
      access_key=" rap1ide/slice_inflate/runs/gd12cq5u",
      id=12,
      fold=1,
   ),
   "20240125__15_12_13_plain-pizza_stage-1_fold-2": dict(
      access_key="rap1ide/slice_inflate/runs/ukoauy9x",
      id=12,
      fold=2,
   ),

   ######## 13 ########
   "20240125__21_15_50_plain-mask_stage-3_fold-0": dict(
      access_key="rap1ide/slice_inflate/runs/sih8wc65",
      id=13,
      fold=0,
   ),
   "20240125__21_15_50_magenta-glass_stage-3_fold-1": dict(
       access_key="rap1ide/slice_inflate/runs/qmvin2ux",
      id=13,
      fold=1,
   ),
   "20240125__21_15_50_leather-plot_stage-3_fold-2": dict(
      access_key="rap1ide/slice_inflate/runs/hvvddqer",
      id=13,
      fold=2,
   ),

   ######## 14 ########
   "20240125__17_37_07_lazy-object_stage-1_fold-0": dict(
      access_key="rap1ide/slice_inflate/runs/e975wi0p",
      id=14,
      fold=0,
   ),
   "20240125__17_37_07_wide-cube_stage-1_fold-1": dict(
      access_key="rap1ide/slice_inflate/runs/6plcdxdr",
      id=14,
      fold=1,
   ),
   "20240125__17_37_07_active-caramel_stage-1_fold-2": dict(
      access_key="rap1ide/slice_inflate/runs/w0bgwxn3",
      id=14,
      fold=2,
   ),

   ######## 15 ########
   "dummy45": dict(
      access_key=DUMMY_RUN,
      id=15,
      fold=0,
   ),
   "dummy46": dict(
      access_key=DUMMY_RUN,
      id=15,
      fold=1,
   ),
   "dummy47": dict(
      access_key=DUMMY_RUN,
      id=15,
      fold=2,
   ),
}


# Define run paths of wandb for MRXCAT

In [ ]:
mrxcat_run_path_list = list(reversed([
    'rap1ide/slice_inflate/runs/egchj2m2',
    'rap1ide/slice_inflate/runs/4fe94wtl',
    'rap1ide/slice_inflate/runs/ihc5g4bg',
    'rap1ide/slice_inflate/runs/wnfu8s1l',
    'rap1ide/slice_inflate/runs/4j1oridg',
    'rap1ide/slice_inflate/runs/lc6itz1v',
    'rap1ide/slice_inflate/runs/t0jx7ix5',
    'rap1ide/slice_inflate/runs/qz5w3pcl',
    'rap1ide/slice_inflate/runs/g4xbtm4v',
    'rap1ide/slice_inflate/runs/ghja6j9o',
    'rap1ide/slice_inflate/runs/ofhiy0sq',
    'rap1ide/slice_inflate/runs/8w171q1e',
]))

In [ ]:
mrxcat_run_name_list = list(reversed([
  '20240126__02_45_48_vibrant-mile_stage-3_fold-0',
  '20240125__22_24_43_staccato-specialist_stage-3_fold-0',
  '20240125__21_31_26_nervous-change_stage-4_fold-0',
  '20240125__18_43_41_lean-pinot_stage-1_fold-0',
  '20240125__18_03_05_each-practice_stage-1_fold-0',
  '20240125__17_21_39_approximate-seat_stage-1_fold-0',
  '20240125__16_46_51_refined-roman_stage-1_fold-0',
  '20240125__16_16_05_unproductive-trainer_stage-1_fold-0',
  '20240125__15_35_02_brass-cupola_stage-1_fold-0',
  '20240125__15_02_28_staccato-retail_stage-1_fold-0',
  '20240125__14_32_54_dry-berry_stage-1_fold-0',
  '20240125__14_15_08_grilled-shape_stage-1_fold-0',
]))

### Print run lists for convenience MRXCAT

In [ ]:
for c_id, (nc, pc) in enumerate(zip(chunks(mrxcat_run_name_list, 1), chunks(mrxcat_run_path_list, 1))):
    print(f"## {c_id}")
    for n, p in zip(nc, pc):
        print(n, "\t", p)
print()

In [ ]:
mrxcat_run_dict = {
   ######## 00 ########
   "20240125__14_15_08_grilled-shape_stage-1_fold-0": dict(
      access_key="rap1ide/slice_inflate/runs/8w171q1e",
      id=0,
      fold=0,
   ),

   ######## 01 ########
   "20240125__14_32_54_dry-berry_stage-1_fold-0": dict(
      access_key="rap1ide/slice_inflate/runs/ofhiy0sq",
      id=1,
      fold=0,
   ),

   ######## 02 ########
   "20240125__15_02_28_staccato-retail_stage-1_fold-0": dict(
      access_key="rap1ide/slice_inflate/runs/ghja6j9o",
      id=2,
      fold=0,
   ),

   ######## 03 ########
   "20240125__15_35_02_brass-cupola_stage-1_fold-0": dict(
      access_key="rap1ide/slice_inflate/runs/g4xbtm4v",
      id=3,
      fold=0,
   ),

   ######## 04 ########
   "20240125__16_16_05_unproductive-trainer_stage-1_fold-0": dict(
      access_key="rap1ide/slice_inflate/runs/qz5w3pcl",
      id=4,
      fold=0,
   ),

   ######## 05 ########
   "20240125__16_46_51_refined-roman_stage-1_fold-0": dict(
      access_key="rap1ide/slice_inflate/runs/t0jx7ix5",
      id=5,
      fold=0,
   ),

   ######## 06 ########
   "20240125__17_21_39_approximate-seat_stage-1_fold-0": dict(
      access_key="rap1ide/slice_inflate/runs/lc6itz1v",
      id=6,
      fold=0,
   ),

   ######## 07 ########
   "20240125__18_03_05_each-practice_stage-1_fold-0": dict(
      access_key="rap1ide/slice_inflate/runs/4j1oridg",
      id=7,
      fold=0,
   ),

   ######## 08 ########
   "20240125__18_43_41_lean-pinot_stage-1_fold-0": dict(
      access_key="rap1ide/slice_inflate/runs/wnfu8s1l",
      id=8,
      fold=0,
   ),

   ######## 09 ########
   "20240126__02_45_48_vibrant-mile_stage-3_fold-0": dict(
      access_key="rap1ide/slice_inflate/runs/egchj2m2",
      id=9,
      fold=0,
   ),

   ######## 10 ########
   "20240125__21_31_26_nervous-change_stage-4_fold-0": dict(
      access_key="rap1ide/slice_inflate/runs/ihc5g4bg",
      id=10,
      fold=0,
   ),

   ######## 11 ########
   "20240125__22_24_43_staccato-specialist_stage-3_fold-0": dict(
      access_key="rap1ide/slice_inflate/runs/4fe94wtl",
      id=11,
      fold=0,
   ),

   ######## 12 ########
   "20240126__15_59_14_navy-goat_stage-4_fold-0": dict(
      access_key="rap1ide/slice_inflate/etommcuz",
      id=12,
      fold=0,
   ),

   ######## 13 ########
   "dummy39": dict(
      access_key=DUMMY_RUN,
      id=13,
      fold=0,
   ),

   ######## 14 ########
   "20240126__16_26_42_future-stable_fold-0_stage-4": dict(
      access_key="rap1ide/slice_inflate/30v1u588",
      id=14,
      fold=0,
   ),

   ######## 15 ########
   "dummy45": dict(
      access_key=DUMMY_RUN,
      id=15,
      fold=0,
   ),
}

# Build latex tables

In [ ]:
def get_wanted_keys(phase):
    assert phase in ['val', 'test']

    wanted_keys = {
        '_id': '_id',
        'prescan_res': 'Precsan props.',
        # 'description': 'Description',
        'prescan_type': 'Prescan props.',
        'slice_res': 'Slice resolution',
        'slice_type': 'Slice props.',
        'first_view': 'Slice view(s)',
        'second_view': 'Second view',
        'fold': 'fold',
        f'scores/{phase}_mean_dice_MYO': 'MYO',
        f'scores/{phase}_mean_dice_LV': 'LV',
        f'scores/{phase}_mean_dice_RV': 'RV',
        f'scores/{phase}_mean_dice_LA': 'LA',
        f'scores/{phase}_mean_dice_RA': 'RA',

        f'scores/{phase}_mean_oa_exclude_bg_dice': '\multicolumn{1}{c}{$\mu\pm\sigma$ }',
        f'scores/{phase}_std_oa_exclude_bg_dice': 'N/A',

        f'scores/{phase}_mean_hd95_MYO': 'MYO',
        f'scores/{phase}_mean_hd95_LV': 'LV',
        f'scores/{phase}_mean_hd95_RV': 'RV',
        f'scores/{phase}_mean_hd95_LA': 'LA',
        f'scores/{phase}_mean_hd95_RA': 'RA',
        f'scores/{phase}_mean_oa_exclude_bg_hd95': '\multicolumn{1}{c}{$\mu\pm\sigma$  }',
        f'scores/{phase}_std_oa_exclude_bg_hd95': 'N/A',

        # 'scores/val_mean_oa_exclude_bg_iou': 'IOU',
        # 'scores/val_std_oa_exclude_bg_iou': '',

        f'scores/{phase}_mean_delta_vol_rel_LV': '\multicolumn{1}{c}{$\mu\pm\sigma$}   ',
        f'scores/{phase}_std_delta_vol_rel_LV': '',
    }
    return wanted_keys

# Build latex table for MMWHS

In [ ]:
df = pd.DataFrame()

for run_idx, (wandb_run_name, rr) in enumerate(mmwhs_run_dict.items()):
   run_key = rr['access_key']
   run_numeric_id = rr['id']
   try:
      wrun = api.run(run_key)
   except:
      print(f"Failed to fetch run {run_key}")
      continue

   run_frame = pd.DataFrame(wrun.history())
   run_frame = run_frame.iloc[-1:]
   run_frame.index = [run_idx]

   if 'dummy' in wandb_run_name:
      run_frame = run_frame.map(lambda x: 0)

   if 3 <= run_numeric_id < 9:
      run_settings = settings['3:9']
   else:
      run_settings = settings[f"{run_numeric_id}:{run_numeric_id+1}"]

   run_frame.insert(0, '_id', [rr['id']])
   run_frame.insert(1, 'description', [run_settings['description']])
   run_frame.insert(2, 'first_view', run_settings['first_view'])
   run_frame.insert(3, 'second_view', run_settings['second_view'])
   run_frame.insert(4, 'prescan_res', run_settings['prescan_res'])
   run_frame.insert(5, 'prescan_type', run_settings['prescan_type'])
   run_frame.insert(6, 'slice_res', run_settings['slice_res'])
   run_frame.insert(7, 'slice_type', run_settings['slice_type'])
   run_frame.insert(8, 'fold', rr['fold'])

   df = pd.concat([df,run_frame])

df

In [ ]:
filtered_frame = df.copy()
filtered_frame = filtered_frame.groupby('_id').agg(get_agg_dict(filtered_frame))

wanted_keys = get_wanted_keys('val')

In [ ]:
# Prepare values
filtered_frame = filtered_frame[wanted_keys.keys()]

percent_keys = [k for k in wanted_keys if "dice" in k or "vol_rel" in k]
mean_keys = [k for k in wanted_keys if "mean" in k]
min_metrics_keys = [k for k in wanted_keys if ("delta" in k or "hd" in k) and not 'std' in k]
max_metrics_keys = [k for k in wanted_keys if ("dice" in k or "iou" in k) and not 'std' in k]
filtered_frame.loc[:,percent_keys] *= 100.

# Aggregate random runs to build mean
random_runs = list(range(0,3)) + 6 * ['is_random'] + list(range(9,16))
filtered_frame.insert(1, 'is_random', random_runs)
filtered_frame = filtered_frame.groupby('is_random', as_index=False).agg(get_agg_dict(filtered_frame))

reindex_idx = list(range(0,3)) + [-1] + list(range(3,10))
filtered_frame = filtered_frame.iloc[reindex_idx,:]
filtered_frame.drop('is_random', axis=1, inplace=True)

filtered_frame

In [ ]:
# Get bold values (best)
group_ranges = [
    [0,5],
    [5,7],
    [7,9],
    [9,11],
]

bold_idxs = []
for sub_range in group_ranges:
    sub_frame = filtered_frame.iloc[sub_range[0]:sub_range[1]]
    optimal_vals_maximize = sub_frame[max_metrics_keys].idxmax()
    optimal_vals_minimize = sub_frame[min_metrics_keys].idxmin()

    bold_idxs.append(optimal_vals_maximize)
    bold_idxs.append(optimal_vals_minimize)

In [ ]:
# Round and convert
filtered_frame = filtered_frame.round(decimals=1)
string_frame = filtered_frame.copy().astype(str)

# Fuse mean and std columns
for m in mean_keys:
    mean_col = string_frame[m].astype(str)
    std_col_key = m.replace("mean","std")
    if std_col_key in string_frame.columns:
        std_col = string_frame[std_col_key].astype(str)
        string_frame[m] = mean_col + r" \pm " + std_col
        string_frame.drop(std_col_key, axis=1, inplace=True)

# Add bold to optimal values
for bold_group in bold_idxs:
    for col_name, idx in zip(bold_group.index, bold_group.values):
        row_idx = string_frame.index.get_loc(idx)
        col_idx = string_frame.columns.get_loc(col_name)
        string_frame.iloc[row_idx,col_idx] = r"\B " + string_frame.iloc[row_idx,col_idx]

# Join first and second view
string_frame = join_cols(string_frame, 'first_view', 'second_view', join_str=r"+")
string_frame = join_cols(string_frame, 'prescan_type', 'prescan_res', join_str=r" ", drop_second=True)
string_frame = join_cols(string_frame, 'slice_type', 'slice_res', join_str=r" ", drop_second=True)

# Replace column names
string_frame.columns = [wanted_keys[c] for c in string_frame.columns]

# Drop and insert spacer columns
string_frame.drop('_id', axis=1, inplace=True)
string_frame.drop('fold', axis=1, inplace=True)

string_frame.insert(3, ' ', len(string_frame)*["\hspace{1pt}"])
string_frame.insert(10, '  ', len(string_frame)*["\hspace{1pt}"])
string_frame.insert(17, '   ', len(string_frame)*["\hspace{1pt}"])

# Add offset to groups in latex
for group_idxs in group_ranges[:-1]:
    txt = string_frame.iloc[group_idxs[1],0]
    string_frame.iloc[group_idxs[1],0] = r"\rule{0pt}{4ex} "+txt

string_frame

In [ ]:
# Save to latex

PM_COL_FORMAT = "S[table-figures-decimal=1,separate-uncertainty=true,table-format=3.1(3)]"
COL_FORMAT = \
    ("c" * 9) \
    + PM_COL_FORMAT \
    + ("c" * 6) \
    + PM_COL_FORMAT \
    + "c" \
    + PM_COL_FORMAT
# 19 cols

header = [r'\multicolumn{3}{c}{\textbf{Experiment I}}'] \
    + 2*[None] \
    + [r'\hspace{1pt}'] \
    + [r'\multicolumn{6}{c}{\textbf{Dice in \% $\uparrow$}}'] + 5*[None] \
    + [r'\hspace{1pt}'] \
    + [r'\multicolumn{6}{c}{\textbf{HD95 in mm $\downarrow$}}'] + 5*[None] \
    + [r'\hspace{1pt}'] \
    + [r'\textbf{$\Delta$vol LV in \% $\downarrow$}']

latex_frame = pd.concat([pd.DataFrame(header, index=string_frame.columns).T, string_frame])

latex_frame.to_latex(
    buf="mmwhs_results.txt",
    escape=False,
    column_format=COL_FORMAT,
    index=False,
)

# Load latex file and replace & NaN with &
with open("mmwhs_results.txt", "r") as f:
    lines = f.readlines()

    lines = [l.replace("& NaN", "") for l in lines]

with open("mmwhs_results.txt", "w") as f:
    f.writelines(lines)

# Insert 5th line before third line
with open("mmwhs_results.txt", "r") as f:
    lines = f.readlines()
    lines.insert(2, lines[4])
    del lines[5]
with open("mmwhs_results.txt", "w") as f:
    f.writelines(lines)

# Build latex table for MRXCAT

In [ ]:
df = pd.DataFrame()

for run_idx, (wandb_run_name, rr) in enumerate(mrxcat_run_dict.items()):
   run_key = rr['access_key']
   run_numeric_id = rr['id']
   try:
      wrun = api.run(run_key)
   except:
      print(f"Failed to fetch run {run_key}")
      continue

   run_frame = pd.DataFrame(wrun.history())
   run_frame = run_frame.iloc[-1:]
   run_frame.index = [run_idx]

   if 'dummy' in wandb_run_name:
      run_frame = run_frame.map(lambda x: 0)

   if 3 <= run_numeric_id < 9:
      run_settings = settings['3:9']
   else:
      run_settings = settings[f"{run_numeric_id}:{run_numeric_id+1}"]

   run_frame.insert(0, '_id', [rr['id']])
   run_frame.insert(1, 'description', [run_settings['description']])
   run_frame.insert(2, 'first_view', run_settings['first_view'])
   run_frame.insert(3, 'second_view', run_settings['second_view'])
   run_frame.insert(4, 'prescan_res', run_settings['prescan_res'])
   run_frame.insert(5, 'prescan_type', run_settings['prescan_type'])
   run_frame.insert(6, 'slice_res', run_settings['slice_res'])
   run_frame.insert(7, 'slice_type', run_settings['slice_type'])
   run_frame.insert(8, 'fold', rr['fold'])

   df = pd.concat([df,run_frame])

df

In [ ]:
filtered_frame = df.copy()
filtered_frame = filtered_frame.groupby('_id').agg(get_agg_dict(filtered_frame))

wanted_keys = get_wanted_keys('test') # Use test here for MRXCAT

In [ ]:
# Prepare values
filtered_frame = filtered_frame[wanted_keys.keys()]

percent_keys = [k for k in wanted_keys if "dice" in k or "vol_rel" in k]
mean_keys = [k for k in wanted_keys if "mean" in k]
min_metrics_keys = [k for k in wanted_keys if ("delta" in k or "hd" in k) and not 'std' in k]
max_metrics_keys = [k for k in wanted_keys if ("dice" in k or "iou" in k) and not 'std' in k]
filtered_frame.loc[:,percent_keys] *= 100.

# Aggregate random runs to build mean
random_runs = list(range(0,3)) + 6 * ['is_random'] + list(range(9,16))
filtered_frame.insert(1, 'is_random', random_runs)
filtered_frame = filtered_frame.groupby('is_random', as_index=False).agg(get_agg_dict(filtered_frame))

reindex_idx = list(range(0,3)) + [-1] + list(range(3,10))
filtered_frame = filtered_frame.iloc[reindex_idx,:]
filtered_frame.drop('is_random', axis=1, inplace=True)

filtered_frame

In [ ]:
# Get bold values (best)
group_ranges = [
    [0,5],
    [5,7],
    [7,9],
    [9,11],
]

bold_idxs = []
for sub_range in group_ranges:
    sub_frame = filtered_frame.iloc[sub_range[0]:sub_range[1]]
    optimal_vals_maximize = sub_frame[max_metrics_keys].idxmax()
    optimal_vals_minimize = sub_frame[min_metrics_keys].idxmin()

    bold_idxs.append(optimal_vals_maximize)
    bold_idxs.append(optimal_vals_minimize)

In [ ]:
# Round and convert
filtered_frame = filtered_frame.round(decimals=1)
string_frame = filtered_frame.copy().astype(str)

# Fuse mean and std columns
for m in mean_keys:
    mean_col = string_frame[m].astype(str)
    std_col_key = m.replace("mean","std")
    if std_col_key in string_frame.columns:
        std_col = string_frame[std_col_key].astype(str)
        string_frame[m] = mean_col + r" \pm " + std_col
        string_frame.drop(std_col_key, axis=1, inplace=True)

# Add bold to optimal values
for bold_group in bold_idxs:
    for col_name, idx in zip(bold_group.index, bold_group.values):
        row_idx = string_frame.index.get_loc(idx)
        col_idx = string_frame.columns.get_loc(col_name)
        string_frame.iloc[row_idx,col_idx] = r"\B " + string_frame.iloc[row_idx,col_idx]

# Join first and second view
string_frame = join_cols(string_frame, 'first_view', 'second_view', join_str=r"+")
string_frame = join_cols(string_frame, 'prescan_type', 'prescan_res', join_str=r" ", drop_second=True)
string_frame = join_cols(string_frame, 'slice_type', 'slice_res', join_str=r" ", drop_second=True)

# Replace column names
string_frame.columns = [wanted_keys[c] for c in string_frame.columns]

# Drop and insert spacer columns
string_frame.drop('_id', axis=1, inplace=True)
string_frame.drop('fold', axis=1, inplace=True)

string_frame.insert(3, ' ', len(string_frame)*["\hspace{1pt}"])
string_frame.insert(10, '  ', len(string_frame)*["\hspace{1pt}"])
string_frame.insert(17, '   ', len(string_frame)*["\hspace{1pt}"])

# Add offset to groups in latex
for group_idxs in group_ranges[:-1]:
    txt = string_frame.iloc[group_idxs[1],0]
    string_frame.iloc[group_idxs[1],0] = r"\rule{0pt}{4ex} "+txt

string_frame

In [ ]:
# Save to latex

PM_COL_FORMAT = "S[table-figures-decimal=1,separate-uncertainty=true,table-format=3.1(3)]"
COL_FORMAT = \
    ("c" * 9) \
    + PM_COL_FORMAT \
    + ("c" * 6) \
    + PM_COL_FORMAT \
    + "c" \
    + PM_COL_FORMAT
# 19 cols

header = [r'\multicolumn{3}{c}{\textbf{Experiment II}}'] \
    + 2*[None] \
    + [r'\hspace{1pt}'] \
    + [r'\multicolumn{6}{c}{\textbf{Dice in \% $\uparrow$}}'] + 5*[None] \
    + [r'\hspace{1pt}'] \
    + [r'\multicolumn{6}{c}{\textbf{HD95 in mm $\downarrow$}}'] + 5*[None] \
    + [r'\hspace{1pt}'] \
    + [r'\textbf{$\Delta$vol LV in \% $\downarrow$}']

latex_frame = pd.concat([pd.DataFrame(header, index=string_frame.columns).T, string_frame])

latex_frame.to_latex(
    buf="mrxcat_results.txt",
    escape=False,
    column_format=COL_FORMAT,
    index=False,
)

# Load latex file and replace & NaN with &
with open("mrxcat_results.txt", "r") as f:
    lines = f.readlines()

    lines = [l.replace("& NaN", "") for l in lines]

with open("mrxcat_results.txt", "w") as f:
    f.writelines(lines)

# Insert 5th line before third line
with open("mrxcat_results.txt", "r") as f:
    lines = f.readlines()
    lines.insert(2, lines[4])
    del lines[5]
with open("mrxcat_results.txt", "w") as f:
    f.writelines(lines)